<a href="https://colab.research.google.com/github/kaiosatiro/datascienceportfolio/blob/master/AvaliacaoAED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#Comparativo dos indices de desenvolvimento humano do municipio de São Paulo por regiões, subprefeituras e distritos. Ano de 2010

#Tratamento

In [354]:
import urllib.request
import zipfile
from io import TextIOWrapper
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import pearsonr


# Fonte de dados:
#               Base de dados (deinfoidhudh20002010dados):
#               http://dados.prefeitura.sp.gov.br/dataset/indice-de-desenvolvimento-humano-municipal/resource/8d7de997-68ae-4d0b-a383-d594ca48ec71
#               Dicionario (DEINFO_IDH_UDH_2000_2010_Variáveis):
#               http://dados.prefeitura.sp.gov.br/dataset/indice-de-desenvolvimento-humano-municipal/resource/01a00699-3cb4-4ea8-887f-6b065ae55960


urlDicionario = 'http://dados.prefeitura.sp.gov.br/dataset/7fe5fa32-4845-4f49-ad1b-81df5336d465/resource/01a00699-3cb4-4ea8-887f-6b065ae55960/download/arquivo-variaveis.zip'
urlBaseDeDados = 'http://dados.prefeitura.sp.gov.br/dataset/7fe5fa32-4845-4f49-ad1b-81df5336d465/resource/8d7de997-68ae-4d0b-a383-d594ca48ec71/download/deinfoidhudh20002010dados.csv'

# --------------Download do zip contendo o dicionario-----
manipulador, _ = urllib.request.urlretrieve(urlDicionario)
zip = zipfile.ZipFile(manipulador, 'r')
arquivo = zip.namelist()[1]
fl = zip.open(arquivo)
# --------------

#Leitura dos csv's
arq = pd.read_csv(urlBaseDeDados, encoding='latin1')
arq2 = pd.read_csv(TextIOWrapper(fl, 'latin1'))

# Tratamento dos dados
lista = ['NOME_UDH', 'ANO', 'IDHM', 'IDHM_E', 'T_SUPER25M', 'T_ANALF25M', 'T_MED25M',
         'IDHM_R', 'RDPC', 'PESOTOT', 'DISTRITO', 'REGIAO8', 'SUBPREF']

dicionario = dict(arq2.filter(items=['Variável', 'Nome da variável']).query('Variável in @lista').to_numpy())
df = arq.filter(items=lista).query('ANO == 2010').drop(columns=['ANO'])

# Dataframes com e sem index
aggdic = {
    'IDHM':'mean',
    'IDHM_E':'mean',
    'T_SUPER25M':'mean',
    'T_ANALF25M':'mean',
    'T_MED25M':'mean',
    'IDHM_R':'mean',
    'RDPC':'mean',
    'PESOTOT':'sum'
}

dfDist = df.groupby(['DISTRITO', 'REGIAO8', 'SUBPREF'], as_index=False).agg(aggdic).round(3)
dfDistIdx = df.groupby(['DISTRITO', 'REGIAO8', 'SUBPREF']).agg(aggdic)

dfSubP = df.groupby(['REGIAO8', 'SUBPREF'], as_index=False).agg(aggdic).round(3)
dfSubPIdx = df.groupby(['REGIAO8', 'SUBPREF']).agg(aggdic)

dfReg = df.groupby(['REGIAO8'], as_index=False).agg(aggdic).round(3)
dfRegIdx = df.groupby(['REGIAO8']).agg(aggdic)


In [355]:
#Tabela das legendas da siglas
values = list(dicionario.keys()), list(dicionario.values())

fig = go.Figure(data=[go.Table(
  columnwidth = [25,100],
  header = dict( values = ['<b>SIGLA</b>', '<b>DESCRIÇÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=12,  height=25))])
fig.update_layout(height=320, width=700, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

#Divisões Regionais

##Graficos

In [373]:
# Grafico 1 IDH, Índice Educacional e Índice de Renda | Por Divisão Regional
fig = px.bar(
    dfReg, x='REGIAO8', y=['IDHM', 'IDHM_R', 'IDHM_E'], 
    height=600, width= 1200, barmode='group', labels=dicionario, text_auto=True, 
    title='IDH, Índice de Renda e Índice Educacional respectivamente 2010 | Por Divisão Regional'
)
fig.show()

In [357]:
# Grafico 2 Renda Per Capita | Por Divisão Regional
fig = px.bar(
    dfReg, x='REGIAO8', y='RDPC', color='REGIAO8', 
    height=600, width= 1200, labels=dicionario, 
    text_auto=True , title='Renda Per Capita 2010 | Por Divisão Regional'
)
# fig.update_layout(xaxis = {"categoryorder":"total descending"})
fig.show()

In [358]:
# Grafico 3 População | Por Divisão Regional 
fig = px.bar(
    dfReg, x='REGIAO8', y='PESOTOT', color='REGIAO8',
    height=600, width= 1200, labels=dicionario, 
    text_auto=True , title='População 2010 | Por Divisão Regional'
)
# fig.update_layout(xaxis = {"categoryorder":"total descending"})
fig.show()

##Tabelas

In [377]:
# Tabela 1 com maiores valores ENTRE Divisões Regionais
values = [
    ['<b>IDH</b>', '<b>Renda per Capita</b>', '<b>Índice de Escolaridade</b>', '<b>População</b>'],
    [dfRegIdx['IDHM'].max().round(3), dfRegIdx['RDPC'].max().round(2), dfRegIdx['IDHM_E'].max().round(3), dfRegIdx['PESOTOT'].max()], 
    [dfRegIdx['IDHM'].idxmax(), dfRegIdx['RDPC'].idxmax(), dfRegIdx['IDHM_E'].idxmax(), dfRegIdx['PESOTOT'].idxmax()]
]

fig = go.Figure(data=[go.Table(
  columnwidth = [25,15,20],
  header = dict( values = ['<b></b>', '<b>VALOR</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=12,  height=25))])
fig.update_layout(height=160, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

In [376]:
# Tabela 2 com menores valores ENTRE Divisões Regionais
values = [
    ['<b>IDH</b>', '<b>Renda per Capita</b>', '<b>Índice de Escolaridade</b>', '<b>População</b>'],
    [dfRegIdx['IDHM'].min().round(3), dfRegIdx['RDPC'].min().round(2), dfRegIdx['IDHM_E'].min().round(3), dfRegIdx['PESOTOT'].min()],
    [dfRegIdx['IDHM'].idxmin(), dfRegIdx['RDPC'].idxmin(), dfRegIdx['IDHM_E'].idxmin(), dfRegIdx['PESOTOT'].idxmin()]
]

fig = go.Figure(data=[go.Table(
  columnwidth = [25,15,20],
  header = dict( values = ['<b></b>', '<b>VALOR</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=12,  height=25))])
fig.update_layout(height=160, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

# Sub Prefeituras

##Graficos

In [365]:
# Grafico 4 Indice de Desenvolvimento Humano | Por Sub Prefeitura
fig = px.bar(
    dfSubP, x='SUBPREF', y=['IDHM'], 
    height=600, width= 1200, color='REGIAO8', 
    labels=dicionario, text_auto=True , title='IDH 2010 | Por Sub Prefeitura'
)
# fig.update_layout(xaxis = {"categoryorder":"total descending"})
fig.show()

In [191]:
# Grafico 5 Renda Per Capita | Por Sub Prefeitura
fig = px.bar(
    dfSubP, x='SUBPREF', y='RDPC', color='REGIAO8', 
    height=600, width= 1200, labels=dicionario, 
    text_auto=True , title='Renda Per Capita 2010 | Por Sub Prefeitura'
)
# fig.update_layout(xaxis = {"categoryorder":"total descending"})
fig.show()

In [196]:
# Grafico 6 População | Por Sub Prefeitura
fig = px.bar(
    dfSubP, x='SUBPREF', y='PESOTOT', color='REGIAO8', 
    height=600, width= 1200, labels=dicionario, 
    text_auto=True , title='População 2010 | Por Sub Prefeitura'
)
# fig.update_layout(xaxis = {"categoryorder":"total descending"})
fig.show()

##Tabelas

In [399]:
# Tabela 3 Maiores valores ENTRE Sub Prefeituras
values = [
    ['<b>IDH</b>', '<b>Renda per Capita</b>', '<b>Índice de Escolaridade</b>', '<b>População</b>'],
    [dfSubPIdx['IDHM'].max().round(3), dfSubPIdx['RDPC'].max().round(2), dfSubPIdx['IDHM_E'].max().round(3), dfSubPIdx['PESOTOT'].max()], 
    [dfSubPIdx['IDHM'].idxmax()[1], dfSubPIdx['RDPC'].idxmax()[1], dfSubPIdx['IDHM_E'].idxmax()[1], dfSubPIdx['PESOTOT'].idxmax()[1]],
    [dfSubPIdx['IDHM'].idxmax()[0], dfSubPIdx['RDPC'].idxmax()[0], dfSubPIdx['IDHM_E'].idxmax()[0], dfSubPIdx['PESOTOT'].idxmax()[0]]
]

fig = go.Figure(data=[go.Table(
  columnwidth = [35,15,25,15],
  header = dict( values = ['<b></b>', '<b>VALOR</b>', '<b>S. PREFEITURA</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=12,  height=25))])
fig.update_layout(height=160, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()


In [414]:
# Tabela 4 Menores valores ENTRE Sub Prefeitura
values = [
    ['<b>IDH</b>', '<b>Renda per Capita</b>', '<b>Índice de Escolaridade</b>', '<b>População</b>'],
    [dfSubPIdx['IDHM'].min().round(3), dfSubPIdx['RDPC'].min().round(2), dfSubPIdx['IDHM_E'].min().round(3), dfSubPIdx['PESOTOT'].min()], 
    [dfSubPIdx['IDHM'].idxmin()[1], dfSubPIdx['RDPC'].idxmin()[1], dfSubPIdx['IDHM_E'].idxmin()[1], dfSubPIdx['PESOTOT'].idxmin()[1]],
    [dfSubPIdx['IDHM'].idxmin()[0], dfSubPIdx['RDPC'].idxmin()[0], dfSubPIdx['IDHM_E'].idxmin()[0], dfSubPIdx['PESOTOT'].idxmin()[0]]
]

fig = go.Figure(data=[go.Table(
  columnwidth = [35,15,25,15],
  header = dict( values = ['<b></b>', '<b>VALOR</b>', '<b>S. PREFEITURA</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left','left'], font_size=12,  height=25))])
fig.update_layout(height=160, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

#Distritos

##IDH

In [203]:
# Grafico 7 IDH | Por Distrito
fig = px.bar(
    dfDist, x='DISTRITO', y=['IDHM'], color='REGIAO8',
    height=600, width= 1800, 
    title='IDH 2010 | Por Distrito'
)
fig.update_layout(xaxis = {"categoryorder":"total descending"})
# fig.update_xaxes(showticklabels=False)
fig.show()

In [410]:
# Tabela 5 - MAIORES IDH's ENTRE Distritos
data = dfDist.sort_values('IDHM', ascending=False).head(5).reset_index()

values = [
    data['IDHM'],
    data['DISTRITO'],
    data['REGIAO8']    
]

fig = go.Figure(data=[go.Table(
  columnwidth = [20,30,20],
  header = dict( values = ['<b>IDH</b>', '<b>DISTRITO</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=14,  height=30))])
fig.update_layout(height=200, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()


In [412]:
# Tabela 6 - MENORES IDH's ENTRE Distritos
data = dfDist.sort_values('IDHM', ascending=True).head(5).reset_index()

values = [
    data['IDHM'],
    data['DISTRITO'],
    data['REGIAO8']    
]

fig = go.Figure(data=[go.Table(
  columnwidth = [20,30,20],
  header = dict( values = ['<b>IDH</b>', '<b>DISTRITO</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=14,  height=30))])
fig.update_layout(height=200, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()


##Renda per Capta

In [206]:
# Grafico 8 Renda Per Capita | Por Distrito
fig = px.bar(
    dfDist, x='DISTRITO', y='RDPC', color='REGIAO8', 
    height=600, width= 1200, 
    title='Renda Per Capita 2010 | Por Distrito'
)
# fig.update_layout(xaxis = {"categoryorder":"total descending"})
fig.update_xaxes(showticklabels=False)
fig.show()

In [448]:
# Tabela 7 - MAIORES Renda Per Capita ENTRE Distritos
data = dfDist.sort_values('RDPC', ascending=False).head(5).reset_index()
dist = data['DISTRITO']
reg = data['REGIAO8']
idh = data['IDHM']

values = [
    data['RDPC'],
    data['DISTRITO'],
    data['REGIAO8']    
]

fig = go.Figure(data=[go.Table(
  columnwidth = [20,30,20],
  header = dict( values = ['<b>Renda Per Capita</b>', '<b>DISTRITO</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=14,  height=30))])
fig.update_layout(height=200, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

In [417]:
# Tabela 8 - MENORES Renda Per Capita ENTRE Distritos
data = dfDist.sort_values('RDPC', ascending=True).head(5).reset_index()
dist = data['DISTRITO']
reg = data['REGIAO8']
idh = data['IDHM']

values = [
    data['RDPC'],
    data['DISTRITO'],
    data['REGIAO8']    
]

fig = go.Figure(data=[go.Table(
  columnwidth = [20,30,20],
  header = dict( values = ['<b>Renda Per Capita</b>', '<b>DISTRITO</b>', '<b>REGIÃO</b>'], align=['left','left'], height=30),
  cells= dict(values= values, align=['left', 'left'], font_size=14,  height=30))])
fig.update_layout(height=200, width=500, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

#Média geral do municipio de São Paulo

In [429]:
#Tabela 9 - Media do IDH, índice educacional e renda per capta
values = [
    ['<b>IDH</b>', '<b>Índice Educacional</b>', '<b>Renda Per Capta</b>'],
    dfDist[['IDHM',	'IDHM_E',	'RDPC']].mean().round(3),
    dfDist[['IDHM',	'IDHM_E',	'RDPC']].std().round(3),
    dfDist[['IDHM',	'IDHM_E',	'RDPC']].min().round(3),
    dfDist[['IDHM',	'IDHM_E',	'RDPC']].max().round(3),

]

fig = go.Figure(data=[go.Table(
  columnwidth = [40,20,20,20],
  header = dict( values = ['<b></b>', '<b>MÉDIA</b>', '<b>DESVIO</b>', '<b>MÍNIMO</b>', '<b>MÁXIMO</b>'], align=['left','left'], height=30, font_size=16),
  cells= dict(values= values, align=['left', 'left'], font_size=18,  height=30))])
fig.update_layout(height=160, width=800, margin=dict(r=5, l=5, t=5, b=5))
fig.show()

#Correlação entre variaveis

##IDH vs Renda

In [449]:
#Grafico correlação 1 IDH vs Renda Per Capta
corr, _ = pearsonr(dfDist['IDHM'], dfDist['RDPC'])
fig = px.scatter(
    dfDist, x ='RDPC', y ='IDHM', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'IDH vs Renda per capita 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

##IDH vs Educação

In [432]:
#Grafico correlação 2 IDH vs 25 anos ou mais com superior completo
corr, _ = pearsonr(dfDist['IDHM'], dfDist['T_SUPER25M'])
fig = px.scatter(
    dfDist, x ='T_SUPER25M', y ='IDHM', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'IDH vs Superior completo >= 25 anos, 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

In [433]:
#Grafico correlação 3 IDH vs 25 anos ou mais com ensino medio completo
corr, _ = pearsonr(dfDist['IDHM'], dfDist['T_MED25M'])
fig = px.scatter(
    dfDist, x ='T_MED25M', y ='IDHM', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'IDH vs Ensino medio completo >= 25 anos, 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

In [434]:
#Grafico correlação 4 IDH vs 25 ano ou mais Taxa analfabetismo
corr, _ = pearsonr(dfDist['IDHM'], dfDist['T_ANALF25M'])
fig = px.scatter(
    dfDist, x ='T_ANALF25M', y ='IDHM', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'IDH vs Taxa de analfabetismo >= 25 anos, 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

## Renda vs Educação

In [450]:
#Grafico correlação 5 Renda per Capta vs Analfabetismo
corr, _ = pearsonr(dfDist['RDPC'], dfDist['T_ANALF25M'])
fig = px.scatter(
    dfDist, x ='T_ANALF25M', y ='RDPC', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'Renda per capita vs Anafalfabetismo >=25 anos, 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

In [451]:
#Grafico correlação 6 Renda per Capta vs Superior completo
corr, _ = pearsonr(dfDist['RDPC'], dfDist['T_SUPER25M'])
fig = px.scatter(
    dfDist, x ='T_SUPER25M', y ='RDPC', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'Renda per caita vs Ensino superior completo >=25 anos, 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

In [453]:
#Grafico correlação 7 Renda per Capta vs Ensino medio completo
corr, _ = pearsonr(dfDist['RDPC'], dfDist['T_MED25M'])
fig = px.scatter(
    dfDist, x ='T_MED25M', y ='RDPC', color='REGIAO8', 
    trendline ='ols', trendline_scope="overall", trendline_color_override='CornflowerBlue',  
    labels=dicionario, title= f'Renda per capita vs Ensino médio completo >=25 anos, 2010 | pearson {corr:.4f}',
    width = 800
)
fig.show()

##Tabelas

In [446]:
#Tabela 10 - Coeficiente de correlação de todas as análises bivariadas 
tabela = [
[
        'IDH x População >= 25 anos com superior completo',
        'IDH x População >= 25 anos com ensino médio completo',
        'IDH x Taxa Analfabetismo da população >= 25 anos',
        'IDH x Renda per capita',
        'Renda per capita x População >= 25 anos com superior completo',
        'Renda per capita x População >= 25 anos com ensino médio completo',
        'Renda per capita x Taxa Analfabetismo da população >= 25 anos'            
],
[
        pearsonr(dfDist['IDHM'], dfDist['T_SUPER25M'])[0].round(4),
        pearsonr(dfDist['IDHM'], dfDist['T_MED25M'])[0].round(4),
        pearsonr(dfDist['IDHM'], dfDist['T_ANALF25M'])[0].round(4),
        pearsonr(dfDist['IDHM'], dfDist['RDPC'])[0].round(4),
        pearsonr(dfDist['RDPC'], dfDist['T_SUPER25M'])[0].round(4),
        pearsonr(dfDist['RDPC'], dfDist['T_MED25M'])[0].round(4),
        pearsonr(dfDist['RDPC'], dfDist['T_ANALF25M'])[0].round(4)
]
          ]
          

fig = go.Figure(data=[go.Table(
  columnwidth = [40,10],
  header = dict( values = ['<b>Par de Variável</b>', '<b>Pearsons</b>'], align=['center','left'], font_size=15),
  cells= dict(values= tabela, align=['center', 'left'], font_size=16,  height=30))])
fig.update_layout(height=290, width=800, margin=dict(r=5, l=5, t=5, b=5))
fig.show()